In [62]:
pip install python-dateutil



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
pip install tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
pip install flask-cors


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [73]:
import requests
from datetime import datetime
from transformers import pipeline
from dateutil import parser

sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# API URLs
NEWS_API_URL = "https://mboum-finance.p.rapidapi.com/v2/markets/news"
REAL_TIME_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/quote"
HISTORY_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/stock/quotes"
# API Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "",  # Replace with your RapidAPI key
    "X-RapidAPI-Host": "mboum-finance.p.rapidapi.com"
}

# Get today's date in the required format (YYYY-MM-DD)
today_date = datetime.now().strftime("%Y-%m-%d")

# Function to get today's financial news
def get_today_financial_news(stock_symbol):
    params = {
        "tickers": stock_symbol,  # Set the stock symbol dynamically
#         "type": "all"             # Optional: Adjust type if needed
    }
    
    print(params)
    
    response = requests.get(NEWS_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        news_data = response.json()
        return news_data
    else:
        print(f"Error fetching news: {response.status_code}")
        return None

# Function to get today's stock price for a specific stock symbol
def get_today_stock_price(stock_symbol):
    params = {
        "symbol": stock_symbol,  # Set the stock symbol dynamically
        "type": "STOCKS"             # Optional: Adjust type if needed
    }
    
    
    response = requests.get(REAL_TIME_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        history_data = response.json()
        return history_data
    else:
        print(f"Error fetching stock price: {response.status_code}")
        return None

def get_history_stock_price(stock_symbol):
    params = {
        "symbol": stock_symbol,  # Set the stock symbol dynamically
    }
    
    
    response = requests.get(HISTORY_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        history_data = response.json()
        return history_data
    else:
        print(f"Error fetching stock price: {response.status_code}")
        return None
    
stock_symbol = "META"  # Replace with the stock symbol you are interested in

# Example: Fetch Financial News for today
news_data = get_today_financial_news(stock_symbol)
today_date = datetime.now().date()

# Initialize vectors for sentiment and confidence
sentiment_vector = []
confidence_vector = []
if news_data:
    print("Financial News for Today:")
#     for article in news_data['body']:
#         print(article)
#         print(f"Title: {article['title']}")
# #         print(f"text: {article['text']}")
#         print("-" * 80)
    
    for article in news_data['body']:
    # Break the loop if the article date is not today
        article_date_str = article.get('time')  # e.g., "Dec 5, 2024, 4:40 AM EST"
        try:
            article_date = parser.parse(article_date_str).date()  # Convert to datetime.date
        except ValueError as e:
            print(f"Error parsing date: {e}")
            continue
        print("article_date: ", article_date)
        print("today_date: ", today_date)
        # Break the loop if the article date is not today
        if article_date != today_date:
            break

    # Get article title
        title = article.get('title')

        # Perform sentiment analysis on the title
        if title:
            sentiment_result = sentiment_pipeline(title)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            # Append sentiment and confidence to respective vectors
            sentiment_vector.append(sentiment)
            confidence_vector.append(confidence)
            # Print the article details and sentiment analysis
            print(f"Title: {title}")
            print(f"Sentiment: {sentiment} (Confidence: {confidence:.2f})")
            print("-" * 80)

sentiment_score = 0
confidence_sum = 0
# Loop through both sentiment and confidence vectors
for i in range(len(sentiment_vector)):
    # Adjust sentiment score based on positive/negative sentiment
    if sentiment_vector[i] == 'positive':
        sentiment_score += 1
    elif sentiment_vector[i] == 'negative':
        sentiment_score -= 1

    # Adjust confidence sum based on corresponding sentiment
    if sentiment_vector[i] == 'positive':
        confidence_sum += confidence_vector[i]
    elif sentiment_vector[i] == 'negative':
        confidence_sum -= confidence_vector[i]
print(confidence_sum)
print(sentiment_score)
# Output the final results
print(f"Cumulative Sentiment Score: {sentiment_vector}")

# Example: Fetch Today's Stock Price for a given stock symbol (e.g., AAPL for Apple)
stock_price_data = get_today_stock_price(stock_symbol)

if stock_price_data:
    print(f"Stock Price for {stock_symbol} on {today_date}:")
#     print(stock_price_data)
    primaryData = stock_price_data['body']['primaryData']
    print(f"price: {primaryData['lastSalePrice']}, volume: {primaryData['volume']}")
    print("-" * 80)
    
    
    
history_price_data = get_history_stock_price(stock_symbol)

if history_price_data:
    print(f"history Stock Price for {stock_symbol} on {today_date}:")
    print(history_price_data)
    entry = history_price_data['body'][0]
    print(f"open: {entry['regularMarketOpen']}, date: {history_price_data['meta']['processedTime']}, recommend: {entry['averageAnalystRating']}")
    print("-" * 80)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'tickers': 'META'}
Financial News for Today:
article_date:  2024-12-05
today_date:  2024-12-05
Title: MicroStrategy Stock Soars. Here's How Much Its Bet on Crypto Bitcoin Is Now Worth.
Sentiment: neutral (Confidence: 0.85)
--------------------------------------------------------------------------------
article_date:  2024-12-05
today_date:  2024-12-05
Title: Nvidia Stock Slips. What Musk, Meta's Huge Data Center Plans Mean for AI Chip Demand.
Sentiment: negative (Confidence: 0.85)
--------------------------------------------------------------------------------
article_date:  2024-12-05
today_date:  2024-12-05
Title: He Was Going to Save Intel. He Destroyed $150 Billion of Value Instead.
Sentiment: neutral (Confidence: 0.83)
--------------------------------------------------------------------------------
article_date:  2024-12-05
today_date:  2024-12-05
Title: Vodafone-Three Merger Gets Conditional U.K. Clearance
Sentiment: neutral (Confidence: 0.72)
-----------------------------------

Title: Avoid MicroStrategy stock and buy MSTX and MSTU ETFs instead
Sentiment: neutral (Confidence: 0.95)
--------------------------------------------------------------------------------
article_date:  2024-12-05
today_date:  2024-12-05
Title: Intel says its 'no wafer left behind' days are over
Sentiment: neutral (Confidence: 0.76)
--------------------------------------------------------------------------------
article_date:  2024-12-04
today_date:  2024-12-05
-0.09801650047302246
1
Cumulative Sentiment Score: ['neutral', 'negative', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral']
Stock Price for META on 2024-12-05:
price: $613.80

In [79]:
import requests
from datetime import datetime
from transformers import pipeline
from dateutil import parser

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# API URLs
NEWS_API_URL = "https://mboum-finance.p.rapidapi.com/v2/markets/news"
REAL_TIME_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/quote"
HISTORY_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/stock/quotes"

# API Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "",  # Replace with your RapidAPI key
    "X-RapidAPI-Host": "mboum-finance.p.rapidapi.com"
}

# Function to fetch today's financial news for a stock symbol
def get_today_financial_news(stock_symbol):
    params = {"tickers": stock_symbol}
    response = requests.get(NEWS_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching news: {response.status_code}")
        return None

# Function to fetch today's stock price for a stock symbol
def get_today_stock_price(stock_symbol):
    params = {"symbol": stock_symbol, "type": "STOCKS" }
    response = requests.get(REAL_TIME_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching stock price: {response.status_code}")
        return None

# Function to fetch historical stock data
def get_history_stock_price(stock_symbol):
    params = {"symbol": stock_symbol}
    response = requests.get(HISTORY_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching history data: {response.status_code}")
        return None

# Function to analyze sentiment and calculate sentiment score and confidence
def analyze_sentiment(news_data):
    sentiment_vector = []
    confidence_vector = []
    
    today_date = datetime.now().date()

    for article in news_data.get('body', []):
        article_date_str = article.get('time')  # e.g., "Dec 5, 2024, 4:40 AM EST"
        
        try:
            article_date = parser.parse(article_date_str).date()  # Convert to datetime.date
        except ValueError as e:
            print(f"Error parsing date: {e}")
            continue

        if article_date != today_date:  # Only process today's articles
            continue

        title = article.get('title')
        
        if title:
            sentiment_result = sentiment_pipeline(title)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            sentiment_vector.append(sentiment)
            confidence_vector.append(confidence)
    print("sentiment_vector: ", sentiment_vector)
    print("confidence_vector: ", confidence_vector)
    return sentiment_vector, confidence_vector

# Function to calculate cumulative sentiment score and confidence sum
def calculate_sentiment_score(sentiment_vector, confidence_vector):
    sentiment_score = 0
    confidence_sum = 0
    
    for i in range(len(sentiment_vector)):
        if sentiment_vector[i] == 'positive':
            sentiment_score += 1
            confidence_sum += confidence_vector[i]
        elif sentiment_vector[i] == 'negative':
            sentiment_score -= 1
            confidence_sum -= confidence_vector[i]
    print("sentiment_score: ", sentiment_score)
    print("confidence_sum: ", confidence_sum) 
    return sentiment_score, confidence_sum

# Main function that ties everything together
def get_stock_info(stock_symbol):
    # Fetch financial news for the stock
    news_data = get_today_financial_news(stock_symbol)
    if not news_data:
        print("No news data available.")
        return None
    
    # Analyze sentiment from the news data
    sentiment_vector, confidence_vector = analyze_sentiment(news_data)

    # Calculate cumulative sentiment score and confidence sum
    sentiment_score, confidence_sum = calculate_sentiment_score(sentiment_vector, confidence_vector)

    # Fetch today's stock price
    stock_price_data = get_today_stock_price(stock_symbol)
    if stock_price_data:
        print(stock_price_data)
        primary_data = stock_price_data['body']['primaryData']
        last_sale_price = primary_data['lastSalePrice']
        volume = primary_data['volume']
    else:
        last_sale_price, volume = None, None

    # Fetch historical stock data
    history_price_data = get_history_stock_price(stock_symbol)
    if history_price_data:
        print(history_price_data)
        entry = history_price_data['body'][0]
        market_open = entry['regularMarketOpen']
        recommend = entry['averageAnalystRating']
    else:
        market_open, recommend = None, None

    # Return relevant outputs
    return {
        "open": market_open,
        "recommend": recommend,
        "confidence_sum": confidence_sum,
        "sentiment_score": sentiment_score,
        "last_sale_price": last_sale_price,
        "volume": volume
    }

# Example usage
if __name__ == "__main__":
    stock_symbol = "META"  # Replace with the stock symbol you are interested in
    stock_info = get_stock_info(stock_symbol)
    
    if stock_info:
        # Print only the necessary outputs
        print(f"Open: {stock_info['open']}")
        print(f"Recommend: {stock_info['recommend']}")
        print(f"Confidence Sum: {stock_info['confidence_sum']:.2f}")
        print(f"Sentiment Score: {stock_info['sentiment_score']}")
        if stock_info['last_sale_price'] is not None:
            print(f"Last Sale Price: {stock_info['last_sale_price']}")
            print(f"Volume: {stock_info['volume']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


sentiment_vector:  ['negative', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'neutral', 'neutral', 'negative', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'negative', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'neutral']
confidence_vector:  [0.8315001130104065, 0.6999037265777588, 0.49230942130088806, 0.9568250775337219, 0.8537421822547913, 0.8495333790779114, 0.833892822265625, 0.7157696485519409, 0.6061660051345825, 0.9349178075790405, 0.9046213626861572, 0.7182268500328064, 0.8676944375038147, 0.8351644277572632, 0.5703110098838806, 0.6544957756996155, 0.680495023727417, 0.8225917220115662, 0.8161998987197876, 0.7129769921302795, 0.8448449969291687, 0.9141480922698975, 0.9574006795883179, 0.8910250663757324, 0

In [ ]:
import pandas as pd
import pickle
from datetime import datetime

# Function to predict return
def predict_return(stock_symbol, date, open_price, volume, weighted_sentiment_score):
    """
    Predict return based on the inputs using the saved SARIMAX model.

    Args:
        stock_symbol (str): The stock symbol (e.g., 'AAPL', 'TSLA').
        date (str): The date for prediction (format: 'YYYY-MM-DD').
        open_price (float): The open price of the stock.
        volume (int): The volume of the stock.
        sentiment_score (int): Sentiment score (-1, 0, or 1).
        confidence_interval (float): Confidence interval for the sentiment score.

    Returns:
        float: Predicted return for the given inputs.
    """
    # Calculate weighted sentiment score

    # Load the saved SARIMAX model for the stock
    model_path = f'/Users/wenjing/Downloads/Capstone/saved_models/{stock_symbol}_sarimax_model.pkl'
    try:
        with open(model_path, 'rb') as model_file:
            model = pickle.load(model_file)
    except FileNotFoundError:
        raise ValueError(f"Model for stock '{stock_symbol}' not found at {model_path}.")

    # Prepare exogenous variables as a DataFrame
    exog = pd.DataFrame({
        'Date': [datetime.strptime(date, '%Y-%m-%d')],
        'open': [open_price],
        'volume': [volume],
        'weighted_sentiment_score': [weighted_sentiment_score]
    })

    # Predict the return using the loaded SARIMAX model
    predictions = model.get_forecast(steps=1, exog=exog[['open', 'volume', 'weighted_sentiment_score']]).predicted_mean

    # Extract the first prediction
    prediction = predictions.iloc[0]

    return prediction


# Define the prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    stock_symbol = data.get('stock_symbol')
    date = data.get('date')
    open_price = data.get('open_price')
    volume = data.get('volume')
    weighted_sentiment_score = data.get('weighted_sentiment_score')

    # Validate inputs
    if not stock_symbol or not date or open_price is None or volume is None or weighted_sentiment_score is None:
        return jsonify({'error': 'Invalid input. Please provide all required fields.'}), 400

    try:
        # Call the prediction function
        predicted_return = predict_return(
            stock_symbol, date, open_price, volume, weighted_sentiment_score
        )
        return jsonify({'stock_symbol': stock_symbol, 'predicted_return': predicted_return})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [93]:
import requests
from datetime import datetime
from transformers import pipeline
from dateutil import parser
import pandas as pd
import pickle

# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# API URLs
NEWS_API_URL = "https://mboum-finance.p.rapidapi.com/v2/markets/news"
REAL_TIME_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/quote"
HISTORY_API_URL = "https://mboum-finance.p.rapidapi.com/v1/markets/stock/quotes"

# API Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "",  # Replace with your RapidAPI key
    "X-RapidAPI-Host": "mboum-finance.p.rapidapi.com"
}

# Function to fetch today's financial news for a stock symbol
def get_today_financial_news(stock_symbol):
    params = {"tickers": stock_symbol}
    response = requests.get(NEWS_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching news: {response.status_code}")
        return None

# Function to fetch today's stock price for a stock symbol
def get_today_stock_price(stock_symbol):
    params = {"symbol": stock_symbol, "type": "STOCKS" }
    response = requests.get(REAL_TIME_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching stock price: {response.status_code}")
        return None

# Function to fetch historical stock data
def get_history_stock_price(stock_symbol):
    params = {"symbol": stock_symbol}
    response = requests.get(HISTORY_API_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching history data: {response.status_code}")
        return None

# Function to analyze sentiment and calculate sentiment score and confidence
def analyze_sentiment(news_data):
    sentiment_vector = []
    confidence_vector = []
    
    today_date = datetime.now().date()

    for article in news_data.get('body', []):
        article_date_str = article.get('time')  # e.g., "Dec 5, 2024, 4:40 AM EST"
        
        try:
            article_date = parser.parse(article_date_str).date()  # Convert to datetime.date
        except ValueError as e:
            print(f"Error parsing date: {e}")
            continue

        if article_date != today_date:  # Only process today's articles
            continue

        title = article.get('title')
        
        if title:
            sentiment_result = sentiment_pipeline(title)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            sentiment_vector.append(sentiment)
            confidence_vector.append(confidence)
    
    return sentiment_vector, confidence_vector

# Function to calculate cumulative sentiment score and confidence sum
def calculate_sentiment_score(sentiment_vector, confidence_vector):
    sentiment_score = 0
    confidence_sum = 0
    
    for i in range(len(sentiment_vector)):
        if sentiment_vector[i] == 'positive':
            sentiment_score += 1
            confidence_sum += confidence_vector[i]
        elif sentiment_vector[i] == 'negative':
            sentiment_score -= 1
            confidence_sum -= confidence_vector[i]
    print("sentiment_score: ", sentiment_score)
    print("confidence_sum: ", confidence_sum) 
    return sentiment_score, confidence_sum

# Function to predict return
def predict_return(stock_symbol, date, open_price, volume, weighted_sentiment_score):
    """
    Predict return based on the inputs using the saved SARIMAX model.

    Args:
        stock_symbol (str): The stock symbol (e.g., 'AAPL', 'TSLA').
        date (str): The date for prediction (format: 'YYYY-MM-DD').
        open_price (float): The open price of the stock.
        volume (int): The volume of the stock.
        weighted_sentiment_score (float): Weighted sentiment score.

    Returns:
        float: Predicted return for the given inputs.
    """
    # Load the saved SARIMAX model for the stock
    model_path = f'/Users/wenjing/Downloads/Capstone/saved_models/{stock_symbol}_sarimax_model.pkl'
    try:
        with open(model_path, 'rb') as model_file:
            model = pickle.load(model_file)
    except FileNotFoundError:
        raise ValueError(f"Model for stock '{stock_symbol}' not found at {model_path}.")

    print("date: ", date)
    print("open_price: ", type(open_price))
#     volume = int(volume.replace(",", ""))

    print("volume: ", type(volume))
    print("weighted_sentiment_score: ", type(weighted_sentiment_score))
    # Prepare exogenous variables as a DataFrame
    exog = pd.DataFrame({
        'Date': [datetime.strptime(date, '%Y-%m-%d')],
        'open': [open_price],
        'volume': [volume],
        'weighted_sentiment_score': [weighted_sentiment_score]
    })
    
    print("exog: ", exog)

    # Predict the return using the loaded SARIMAX model
    predictions = model.get_forecast(steps=1, exog=exog[['open', 'volume', 'weighted_sentiment_score']]).predicted_mean

    # Extract the first prediction
    prediction = predictions.iloc[0]

    return prediction

# Main function that ties everything together
def get_stock_info(stock_symbol):
    # Fetch financial news for the stock
    news_data = get_today_financial_news(stock_symbol)
    if not news_data:
        print("No news data available.")
        return None
    
    # Analyze sentiment from the news data
    sentiment_vector, confidence_vector = analyze_sentiment(news_data)

    # Calculate cumulative sentiment score and confidence sum
    sentiment_score, confidence_sum = calculate_sentiment_score(sentiment_vector, confidence_vector)

    # Fetch today's stock price
    stock_price_data = get_today_stock_price(stock_symbol)
    if stock_price_data:
        primary_data = stock_price_data['body']['primaryData']
        last_sale_price = primary_data['lastSalePrice']
    else:
        last_sale_price = None, None

    # Fetch historical stock data
    history_price_data = get_history_stock_price(stock_symbol)
    if history_price_data:
        entry = history_price_data['body'][0]
        market_open = entry['regularMarketOpen']
        recommend = entry['averageAnalystRating']
        volume = entry['regularMarketVolume']
    else:
        market_open, recommend, volumn = None, None, None

    # Calculate weighted sentiment score (based on confidence sum)
    weighted_sentiment_score = confidence_sum

    # Predict return
    date = datetime.now().strftime('%Y-%m-%d')
    print("stock_symbol: ", stock_symbol)
    print("date: ", date)
    print("market_open: ", market_open)
    print("volume: ", volume)
    print("weighted_sentiment_score: ", weighted_sentiment_score)
    predicted_return = predict_return(stock_symbol, date, market_open, volume, weighted_sentiment_score)

    # Return relevant outputs
    return {
        "predicted_return": predicted_return,
        "recommend": recommend,
        "last_sale_price": last_sale_price,
        "weighted_sentiment_score": weighted_sentiment_score
    }

# Example usage
if __name__ == "__main__":
    stock_symbol = "GOOG"  # Replace with the stock symbol you are interested in
    stock_info = get_stock_info(stock_symbol)
    
    if stock_info:
        # Print only the necessary outputs
        print(f"Predicted Return: {stock_info['predicted_return']}")
        print(f"Recommend: {stock_info['recommend']}")
        print(f"Last Sale Price: {stock_info['last_sale_price']}")


/Users/wenjing/miniforge3/envs/w210/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/wenjing/miniforge3/envs/w210/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


sentiment_score:  -1
confidence_sum:  -2.0961392521858215
stock_symbol:  GOOG
date:  2024-12-05
market_open:  172.78
volume:  17334377
weighted_sentiment_score:  -2.0961392521858215
date:  2024-12-05
open_price:  <class 'float'>
volume:  <class 'int'>
weighted_sentiment_score:  <class 'float'>
exog:          Date    open    volume  weighted_sentiment_score
0 2024-12-05  172.78  17334377                 -2.096139
Predicted Return: 0.00015291402685435086
Recommend: 1.6 - Buy
Last Sale Price: $175.61
